In [2]:
pi = {
    'ip' : '192.168.0.201',
    'user' : 'user',
    'running on' : 'http://192.168.0.201:5000/'}
jetson = {
    'ip' : '192.168.0.171',
    'user' : 'user',
    'running on' : 'http://192.168.0.171:5000/'}

In [3]:
with open("runModel.sh","r") as f:
    print(f.read())

#!/bin/bash

echo "Running model..."

source base/bin/activate
cd $HOME/Documents/edgeAI/

python3 app.py runs/detect/train/weights/best.pt mittel_1.mp4

echo "Model run complete!"


In [4]:
import paramiko
import time 

def execute_command(command):
    try:
        stdin, stdout, stderr = ssh_client.exec_command(command)
        return stdout.read().decode()
    except Exception as e:
        return f"Error executing command: {e}"

def put_file(ip, user, passwd, source, destination, port=22):
    try:
        ssh_client = paramiko.SSHClient()
        #client.load_system_host_keys()
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh_client.connect(ip, username=user, password=passwd)

        ftp_client = ssh_client.open_sftp()
        ftp_client.put(source, destination)
        ftp_client.close()
        ssh_client.close()
        return 0
    except Exception as e:
        print(e)
        return -1

def ssh_command(ip, user, passwd, command, port=22, timeout=100):
    try:
        ssh_client = paramiko.SSHClient()
        #client.load_system_host_keys()
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh_client.connect(ip, username=user, password=passwd)

        stdin, stdout, stderr = ssh_client.exec_command(command)
        start_time = time.time()
        output = ''
        error = ''
        while True:
            if stdout.channel.recv_ready():
                output += stdout.read().decode()
            if stderr.channel.recv_ready():
                error += stderr.read().decode()
            if stdout.channel.exit_status_ready():
                break
            if time.time() - start_time > timeout:
                print("Timeout reached. Terminating the command.")
                break
            time.sleep(0.1)
        ssh_client.close()
        return 0, output, error
    except Exception as e:
        print(e)
        return -1, output, error

#ssh_command(pi['ip'], pi['user'], 'edge_Pi_123', 'touch ~/Documents/scripts/runModel.sh')
#put_file(pi['ip'], pi['user'], 'edge_Pi_123', 'runModel.sh', '~/Documents/scripts/runModel.sh')
with open("runModel.sh","r") as f:
    cmd = f.read()
    f.close()
#ssh_command(pi['ip'], pi['user'], 'edge_Pi_123', cmd)

In [13]:
from PIL import Image
import requests
import cv2
import numpy as np

class edge_device:
    def __init__(self, ip, user, passwd):
        self.ip = ip
        self.user = user
        self.passwd = passwd
        self.ssh_client = None

    def ssh_open(self):
        ssh_client = paramiko.SSHClient()
        #client.load_system_host_keys()
        ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        ssh_client.connect(self.ip, username=self.user, password=self.passwd)
        self.ssh_client = ssh_client

    def ssh_command(self, command, timeout=60*10):
        if self.ssh_client == None:
            #print("ssh client not open")
            self.ssh_open()
            #print("ssh client opened")
        if isinstance(command, list):
            command = ' && '.join(command)
        stdin, stdout, stderr = self.ssh_client.exec_command(command, timeout=timeout)

        return stdout.read().decode(), stderr.read().decode(),

    def ssh_close(self):
        if self.ssh_client != None:
            self.ssh_client.close()
            self.ssh_client = None

    def get_images(self, sleep = 60):
        time.sleep(sleep)
        cap = cv2.VideoCapture(f'https://{self.ip}:5000')
        while(cap.isOpened()):
            ret, image = cap.read()    
            loadedImage = cv2.imdecode(image, cv2.IMREAD_COLOR)
            cv2.imshow('frame',loadedImage)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()
        return loadedImage

    

def run_model(device, model, video, output = '', source = "'base/bin/activate'"):
    images = []
    try:
        command = []
        command.append(f"source {source}")
        if model == 'ncnn':
            command.append(f'cd ~/Documents/edgeAI/yolov8')
            command.append(f'timeout 5m python3 app.py best_ncnn_model/ {video}' if output == '' else f'timeout 5m python3 app.py best_ncnn_model/ {video} > {output}')
        else:
            command.append(f'cd ~/Documents/edgeAI')
            command.append(f'timeout 5m python3 app.py {model} {video}' if output == '' else f'timeout 2m python3 app.py {model} {video} > {output}')
            try:
                with concurrent.futures.ThreadPoolExecutor() as executor:
                    future_img = executor.submit(device.get_images)
                    future_ssh = executor.submit(device.ssh_command, command)
                    concurrent.futures.wait([future_img, future_ssh])
                    images = future_img.result()
                    stdout, stderr = future_ssh.result()
            except Exception as e:
                print(e)
        #print(stdout, stderr)
        return stdout, stderr, images
    except Exception as e:
        print(e)
        return -1, e, images

pi = edge_device('192.168.0.47', 'user', 'edge_Pi_123')
for i in range(10):
    I = str(i+1)
    #out, err = run_model(pi, 'runs/detect/train/weights/best.pt', f'videos-self-recorded/{I}.mp4', f'piLite2_bestPt_selfRecorded{I}.txt', "'base/bin/activate'")

    #print(out, err)



In [10]:
piUbuntu = edge_device('192.168.0.42', 'user', 'edge_Pi_123')
#x = piUbuntu.get_images()

#x, err, images = run_model(piUbuntu, 'runs/detect/train/weights/best.pt', 'short.mp4', 'piUbuntu_bestPt_short.txt', "'base/bin/activate'")
#print(images[0], len(images))
cap = cv2.VideoCapture('http://192.168.0.42:5000')
while(cap.isOpened()):
    ret, image = cap.read()    
    image = np.asarray(bytearray(image), dtype="uint8")
    loadedImage = cv2.imdecode(image, cv2.IMREAD_COLOR)
    try:
        loadedImage = cv2.cvtColor(loadedImage, cv2.COLOR_BGR2RGB)
    except Exception as e:
        print(e)
    if loadedImage is not None and len(loadedImage) > 0:
        cv2.imshow('frame',loadedImage)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

OpenCV(4.9.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

OpenCV(4.9.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

OpenCV(4.9.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

OpenCV(4.9.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

OpenCV(4.9.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

OpenCV(4.9.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

OpenCV(4.9.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

OpenCV(4.9.0) /io/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.

KeyboardInterrupt: 

In [29]:
response = None
try: 
    response = requests.get(f"http://192.168.0.42:5000", timeout = 5)
except Exception as e:
    print(e)
#print(response.raw.read())
print(response)

HTTPConnectionPool(host='192.168.0.42', port=5000): Read timed out.
None


In [79]:
x = piUbuntu.get_images(0)
print(x)

[tls @ 0x468b600] error:1408F10B:SSL routines:ssl3_get_record:wrong version number


UnboundLocalError: cannot access local variable 'loadedImage' where it is not associated with a value

In [55]:
print(x, err,images)

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.42:5000
Press CTRL+C to quit
192.168.0.168 - - [11/Jul/2024 18:14:34] "GET / HTTP/1.1" 200 -
 []


In [33]:
def cat_file(device, file):
    try:
        command = []
        command.append(f"cat {file}")
        stdout, stderr = device.ssh_command(command)
        return stdout
    except Exception as e:
        print(e)
        return -1, e
print(cat_file(pi, '~/Documents/edgeAI/pi_bestPt_selfRecorded1.txt'))

 * Serving Flask app 'app'
 * Debug mode: off

0: 640x384 12 cars, 1 bus, 5145.3ms
Speed: 37.1ms preprocess, 5145.3ms inference, 18.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 13 cars, 1 bus, 2 traffic signs, 4896.8ms
Speed: 57.2ms preprocess, 4896.8ms inference, 9.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 14 cars, 1 bus, 4906.8ms
Speed: 54.4ms preprocess, 4906.8ms inference, 7.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 12 cars, 1 bus, 4509.5ms
Speed: 58.1ms preprocess, 4509.5ms inference, 8.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 14 cars, 1 bus, 4215.4ms
Speed: 57.2ms preprocess, 4215.4ms inference, 4.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 14 cars, 1 bus, 4787.7ms
Speed: 55.0ms preprocess, 4787.7ms inference, 8.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 pedestrian, 12 cars, 1 bus, 4342.6ms
Speed: 58.2ms preprocess, 4342.6ms inference, 7.7ms postprocess

In [20]:
print(err)

In [ ]:
import subprocess
import os
import time

def execute_command(command, timeout=100):
    try:
        return subprocess.check_output(command, shell=True, executable='/bin/bash', timeout=timeout).decode()
    except Exception as e:
        return f"Error executing command: {e}"
